In [1]:
import pandas as pd
import json
import requests
import urllib.request
import numpy as np
import os
import time

## import all the summaries

In [2]:
path = '../data/20210315_sample/'

# Slaat de bestandsnamen en summaries op als lists
name_list = []
sum_list = []

for f in os.listdir(path):
    if f.endswith('.txt'):
        name_list.append(f[:-4])
        sum_list.append(open(path+f, 'r').read())

In [3]:
# manier om meer te laten zien van een dataframe item.
pd.set_option('display.max_colwidth', 1000)

# Store in dataframe, totale dataset beter in np array dit weglaten?
df = pd.DataFrame()
df['filename'] = name_list
df['summary'] = sum_list
# df.head()

In [4]:

# ts = df[df['filename']== '61FB7787-C915-44EA-B7B2-0047A99ABF12']['summary']
# print("Samenvatting voor de text in volgende cell:")
# print(ts.to_string(index=False))

## import the transcribed text

##  the rigth timeframe to acces the target text to summarize
### 1. store the start and end time with the corresponding summaries

In [5]:
def min_to_ms(timestamp):
    # input timestamp: 01:mm:ss
    # output ms
    ts_split = timestamp.split(':')
    ts_split = list(map(int, ts_split))
    ms = (ts_split[1]*60 + ts_split[2]) *1000
    return ms

In [6]:
# convert start-/endtime to ms[]
df_mani = pd.read_csv(path+'manifest.csv', delimiter =';')
df_mani.columns = ['filename','date', 'starttime', 'endtime', 'jsonurl', 'mediafile']
df_mani['starttime'] = df_mani['starttime'].apply(min_to_ms)
df_mani['endtime'] = df_mani['endtime'].apply(min_to_ms)
# df_mani.head()

In [7]:
# append to df
df_time = pd.merge(df, df_mani, on=['filename'])
df_time.head()

,filename,summary,date,starttime,endtime,jsonurl,mediafile
0,002FFFD8-801A-4232-9AE9-FBD460CE5089,"Wethouders jeugd maken zich ernstig zorgen over de situatie van de kinderen en jongeren in deze lockdown, schrijft trouw. Ze roepen het Rijk op om jongeren weer perspectief te geven met net zoveel urgentie als bedrijven die krijgen. En dat gaat volgens de wethouders verder dan alleen het heropenen van de scholen. Er moeten een actieplan komen.",2021-02-01 08:07,429000,457000,http://zmrecordingstorage.blob.core.windows.net/recordings/2021/01022021/NPO1_NOSJournaal_01022021_0700/46cd1fb8-d562-4db9-8a16-1d31dd40d7f3-spraaklab.json,http://zoommedia.blob.core.windows.net/video/20210201/NPO1_NOSJournaal_01022021_0700/002fffd8-801a-4232-9ae9-fbd460ce5089.m3u8
1,003C931A-BF62-418E-8223-C7BC1DE9698A,De negen Arnhemse villamoord veroordeelden vangen vandaag bot. Volgens hun advocaten zaten ze jarenlang onterecht vast voor moord op een vrouw in 1998. Vorig jaar vertelde één van de veroordelen dat zijn verklaring vals is. Een advies aan de Hoge Raad stelt dat deze geruchtmakende zaak niet over hoeft te worden gedaan. Misdaadverslaggever van De Telegraaf John van den Heuvel zegt dat het heel onrechtvaardig voelt. De advocaat heeft wel gelijk als hij zegt dat de lat erg hoog ligt om een herziening op het vonnis te krijgen. Dat is om een aantal redenen. Gesprek met misdaadverslaggever van De Telegraaf John van den Heuvel.,2021-02-02 19:46,676000,958000,http://zmrecordingstorage.blob.core.windows.net/recordings/2021/02022021/RTL4_Boulevard_02022021_1835/412fc32d-d7d9-4169-b044-69147961d944-spraaklab.json,http://zoommedia.blob.core.windows.net/video/20210202/RTL4_Boulevard_02022021_1835/003c931a-bf62-418e-8223-c7bc1de9698a.m3u8
2,00FE2ED8-A443-43C2-990F-FD3B7D54C1BC,"Uit cijfers van het UWV is de verwachting dat 189.000 banen verloren gaan. Journalist Jean Dohmen zegt daarover dat ondanks de initiatieven van de overheid om geld te pompen in de economie heel wat bedrijven het moeilijk hebben. Bijgevolg is het waarschijnlijk dat de bedrijven die het nu al moeilijk hebben, het straks niet meer gaan redden en er dus faillissementen op de loer liggen. Sommige retailers deden het bijzonder genoeg net beter als voor de crisis, maar dat verschilt erg van de ene zaak tot de andere. Gesprek met Jean Dohmen, journalist bij het Financieele Dagblad.",2021-01-30 18:27,1656000,1770000,http://zmrecordingstorage.blob.core.windows.net/recordings/2021/30012021/Radio1_WNLOpZaterdag_30012021_1700/1aa7ea56-555c-4580-9655-213d061500be-spraaklab.json,http://zoommedia.blob.core.windows.net/video/20210201/Radio1_WNLOpZaterdag_30012021_1700/00fe2ed8-a443-43c2-990f-fd3b7d54c1bc.mp3
3,02319F8F-3F35-4C0F-8F4C-6426D1121908,"Bij de supermarkten Jumbo en Lidl werden het afgelopen jaar het meest gestunt met vlees zonder keurmerk, zegt Wakker Dier tegen RTL nieuws. Bijna tweederde van alle vleesaanbiedingen bij de Jumbo en Lidl was vlees zonder keurmerk. Plus deed het volgens Wakker Dier het beste met 37 procent. Dat dierenwelzijnsorganisatie constateert dat er steeds meer met zogenoemde kilo-knallers wordt geadverteerd.",2021-02-01 07:40,2421000,2452000,http://zmrecordingstorage.blob.core.windows.net/recordings/2021/01022021/Radio1_Radio1Journaal_01022021_0600/ee32dee2-bab9-4255-8f58-86d31902ea0e-spraaklab.json,http://zoommedia.blob.core.windows.net/video/20210201/Radio1_Radio1Journaal_01022021_0600/02319f8f-3f35-4c0f-8f4c-6426d1121908.m3u8
4,027AB628-7528-413B-A636-FC45DEF52CCA,"De verdediging in het MH17-proces wil weten of het Openbaar Ministerie belangrijke wrakstukken van het neergeschoten vliegtuig achterhoudt. De advocaten van verdachte Oleg Poelatov mogen de wrakstukken met een eigen deskundige bekijken. Zij willen onderzoeken of vlucht MH17 door een Oekraïens gevechtsvliegtuig kan zijn neergeschoten, in plaats van door een Russische Buk-raket. De advocaten van verdachte Oleg Poelatov mogen de wrakstukken met een eigen deskundige bekijken. Zij willen onderzoeken of vlucht

### 2. retrieve corresponding transcript based on timestamps

In [10]:
def get_target_text(jsonurl, starttime, endtime):

#     # open de json file
#     with urllib.request.urlopen(jsonurl) as url:
#         data_json = json.loads(url.read().decode())
#         print(url)
    data_json = json.loads(requests.get(jsonurl).text)
        
    trans_text = ""
    for part in data_json:
        for i in range(len(part['result'])):
            if (starttime <= int(part['result'][i][1])) & (int(part['result'][i][2]) <= endtime +1000):
                trans_text += part['result'][i][0] + " "
    return trans_text


In [11]:
# retrieve the transcripts
df_time['target_text'] = df_time.apply(lambda x: get_target_text(x['jsonurl'], x['starttime'], x['endtime']), axis=1)

In [12]:
df_time.head()

,filename,summary,date,starttime,endtime,jsonurl,mediafile,target_text
0,002FFFD8-801A-4232-9AE9-FBD460CE5089,"Wethouders jeugd maken zich ernstig zorgen over de situatie van de kinderen en jongeren in deze lockdown, schrijft trouw. Ze roepen het Rijk op om jongeren weer perspectief te geven met net zoveel urgentie als bedrijven die krijgen. En dat gaat volgens de wethouders verder dan alleen het heropenen van de scholen. Er moeten een actieplan komen.",2021-02-01 08:07,429000,457000,http://zmrecordingstorage.blob.core.windows.net/recordings/2021/01022021/NPO1_NOSJournaal_01022021_0700/46cd1fb8-d562-4db9-8a16-1d31dd40d7f3-spraaklab.json,http://zoommedia.blob.core.windows.net/video/20210201/NPO1_NOSJournaal_01022021_0700/002fffd8-801a-4232-9ae9-fbd460ce5089.m3u8,En dat gaat volgens de wethouders verder dan alleen het heropenen van de scholen. Moeten actieplan komen maar en al twintig wethouders uit kleine en middelgrote gemeente. Ondersteunen dat idee ze eenderde van de Nederlanders wil dat supermarkten stoppen met de verkoop van sigaretten en steunen daarmee dus het kabinetsplan. Dat meldt het ADE op basis van onderzoek zevenenzestig. Procent is voor een verbod per tweeduizend tweeëntwintig. Eenenzeventig procent van hen wordt zelfs al binnen een
1,003C931A-BF62-418E-8223-C7BC1DE9698A,De negen Arnhemse villamoord veroordeelden vangen vandaag bot. Volgens hun advocaten zaten ze jarenlang onterecht vast voor moord op een vrouw in 1998. Vorig jaar vertelde één van de veroordelen dat zijn verklaring vals is. Een advies aan de Hoge Raad stelt dat deze geruchtmakende zaak niet over hoeft te worden gedaan. Misdaadverslaggever van De Telegraaf John van den Heuvel zegt dat het heel onrechtvaardig voelt. De advocaat heeft wel gelijk als hij zegt dat de lat erg hoog ligt om een herziening op het vonnis te krijgen. Dat is om een aantal redenen. Gesprek met misdaadverslaggever van De Telegraaf John van den Heuvel.,2021-02-02 19:46,676000,958000,http://zmrecordingstorage.blob.core.windows.net/recordings/2021/02022021/RTL4_Boulevard_02022021_1835/412fc32d-d7d9-4169-b044-69147961d944-spraaklab.json,http://zoommedia.blob.core.windows.net/video/20210202/RTL4_Boulevard_02022021_1835/003c931a-bf62-418e-8223-c7bc1de9698a.m3u8,Van jou. Jawel. Negen Arnhemse villa moord veroordeelde vanaf vandaag bot. Volgens hun advocaten zaten ze jarenlang onterecht vast voor moord op een vrouw in negentien achtennegentig. Vorig jaar vertelde een van de veroordelen dat zijn verklaring waardoor al deze mannen jarenlang celstraf kreeg. Vals is vies vandaag aan de Hoge Raad stelt dat deze geruchtmakende zaak. Niet over hoeft te worden gedaan de stelling van vandaag. Maar hopelijk denkt Hoogenraad er inhoudelijk anders over. Dat in achtennegentig vindt de politie in een Arnhemse villa twee vrouwen. En drieënzestigjarige vrouw is doodgeschoten en jongere vriendin houdt zich dood maar overleeft. Zei omschrijft de dader als een getinte man met een grote snor. Toch worden er negen verdachten aangehouden en zij. Ontkennen. Ja. Nee. Dan anderen. Maar. Door bekentenis van verdachten worden de mannen veroordeeld tot jarenlange straffen naar zegt. Hij een jaar later zegt maar dat die destijds onder grote druk van de politie liegt. K...
2,00FE2ED8-A443-43C2-990F-FD3B7D54C1BC,"Uit cijfers van het UWV is de verwachting dat 189.000 banen verloren gaan. Journalist Jean Dohmen zegt daarover dat ondanks de initiatieven van de overheid om geld te pompen in de economie heel wat bedrijven het moeilijk hebben. Bijgevolg is het waarschijnlijk dat de bedrijven die het nu al moeilijk hebben, het straks niet meer gaan redden en er dus faillissementen op de loer liggen. Sommige retailers deden het bijzonder genoeg net beter als voor de crisis, maar dat verschilt erg van de ene zaak tot de andere. Gesprek met Jean Dohmen, journalist bij het Financieele Dagblad.",2021-01-30 18:27,1656000,1770000,http://zmrecordingstorage.blob.core.windows.net/recordings/2021/30012021/Radio1_WNLOpZaterdag_300120

In [16]:
#save as CSV file
df_time.to_csv(path_or_buf = '../data/summary_with_target.csv', index=False)
df_cleaned = df_time[['filename','target_text','summary']]
df_cleaned.to_csv(path_or_buf = '../data/summary_with_target_cleaned.csv', index=False)

In [17]:
df_cleaned.head(50)

,filename,target_text,summary
0,002FFFD8-801A-4232-9AE9-FBD460CE5089,En dat gaat volgens de wethouders verder dan alleen het heropenen van de scholen. Moeten actieplan komen maar en al twintig wethouders uit kleine en middelgrote gemeente. Ondersteunen dat idee ze eenderde van de Nederlanders wil dat supermarkten stoppen met de verkoop van sigaretten en steunen daarmee dus het kabinetsplan. Dat meldt het ADE op basis van onderzoek zevenenzestig. Procent is voor een verbod per tweeduizend tweeëntwintig. Eenenzeventig procent van hen wordt zelfs al binnen een,"Wethouders jeugd maken zich ernstig zorgen over de situatie van de kinderen en jongeren in deze lockdown, schrijft trouw. Ze roepen het Rijk op om jongeren weer perspectief te geven met net zoveel urgentie als bedrijven die krijgen. En dat gaat volgens de wethouders verder dan alleen het heropenen van de scholen. Er moeten een actieplan komen."
1,003C931A-BF62-418E-8223-C7BC1DE9698A,Van jou. Jawel. Negen Arnhemse villa moord veroordeelde vanaf vandaag bot. Volgens hun advocaten zaten ze jarenlang onterecht vast voor moord op een vrouw in negentien achtennegentig. Vorig jaar vertelde een van de veroordelen dat zijn verklaring waardoor al deze mannen jarenlang celstraf kreeg. Vals is vies vandaag aan de Hoge Raad stelt dat deze geruchtmakende zaak. Niet over hoeft te worden gedaan de stelling van vandaag. Maar hopelijk denkt Hoogenraad er inhoudelijk anders over. Dat in achtennegentig vindt de politie in een Arnhemse villa twee vrouwen. En drieënzestigjarige vrouw is doodgeschoten en jongere vriendin houdt zich dood maar overleeft. Zei omschrijft de dader als een getinte man met een grote snor. Toch worden er negen verdachten aangehouden en zij. Ontkennen. Ja. Nee. Dan anderen. Maar. Door bekentenis van verdachten worden de mannen veroordeeld tot jarenlange straffen naar zegt. Hij een jaar later zegt maar dat die destijds onder grote druk van de politie liegt. K...,De negen Arnhemse villamoord veroordeelden vangen vandaag bot. Volgens hun advocaten zaten ze jarenlang onterecht vast voor moord op een vrouw in 1998. Vorig jaar vertelde één van de veroordelen dat zijn verklaring vals is. Een advies aan de Hoge Raad stelt dat deze geruchtmakende zaak niet over hoeft te worden gedaan. Misdaadverslaggever van De Telegraaf John van den Heuvel zegt dat het heel onrechtvaardig voelt. De advocaat heeft wel gelijk als hij zegt dat de lat erg hoog ligt om een herziening op het vonnis te krijgen. Dat is om een aantal redenen. Gesprek met misdaadverslaggever van De Telegraaf John van den Heuvel.
2,00FE2ED8-A443-43C2-990F-FD3B7D54C1BC,Goed deze week bleek overigens dat het UWV. Dat vond ik echt een heel slecht cijfer verwachten dat Honderd negenentachtig duizenden banen verloren gaan. Ja het kracht dat ook het interessant is natuurlijk hè we hadden natuurlijk net eigenlijk al over over die. Gratuit zet die grote reset en we hadden het al over de enorme bedragen die in de economie worden gepompt ook in Nederland. Hij ziet er op dit moment eigenlijk die werkloosheid nog relatief laag is en je zou zeggen ja eigenlijk is dat uitzonderlijk we hebben. De grootste hebben grote krimp gezien in twintig twintig. Forse krimpt die we lang niet hebben gezien van die omvang. Maar we zien ook weinig faillissementen en dat allemaal het gevolg van het beleid van de overheid dat op dit moment eigenlijk. De de de de grote failliet gemeenten die zijn historisch laag nou die blijven uit. En ook een sterke oploop van de werkloosheid op enig moment mag je wel aannemen dat er een soort inhaalslag geplaatst vinden. Dat we dus die failli...,"Uit cijfers van het UWV is de verwachting dat 189.000 banen verloren gaan. Journalist Jean Dohmen zegt daarover dat ondanks de initiatieven van de overheid om geld te pompen in de economie heel wat bedrijven het moeilijk hebben. Bijgevolg is het waarschijnlijk dat de bedrijven die het nu al moeilijk hebben, het straks niet meer gaan redden en er dus faillissementen op de loer l